## 2. Data Engineering - Process CSV files into Hive Tables (Parquet)

### Create Spark session with Hive Enabled

Create a Spark session, connect to Hive Metastore and enable Hive support in Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType

warehouse_location = 'gs://<project-id>-demo/hive-warehouse'
service_endpoint = 'thrift://hive-cluster-m.us-central1-f:9083'

from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Spark - Data Eng Demo') \
  .config("hive.metastore.uris", service_endpoint)  \
  .config("spark.sql.warehouse.dir", warehouse_location) \
  .enableHiveSupport() \
  .getOrCreate()

Check the first 1000 bytes of a file on GCS

In [ ]:
!gsutil cat -h -r 0-1000 gs://cloud-ml-tables-data/bank-marketing.csv

==> gs://cloud-ml-tables-data/bank-marketing.csv <==
Age,Job,MaritalStatus,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,PDays,Previous,POutcome,Deposit
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,1
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,1
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,1
58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,1
43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,1
41,admin.,divorced,secondary,

### Get Spark application ID 

This is useful to easily fine application in the Spark History UI

In [ ]:
spark.conf.get("spark.app.id")

'application_1598619717300_0026'

Check what databases are in the Hive Warehouse

In [ ]:
spark.sql("""
SHOW DATABASES;
""").show()

+--------------------+
|           namespace|
+--------------------+
|        bank_demo_db|
|             default|
|house_prices_demo_db|
|     housing_demo_db|
+--------------------+



In [ ]:
spark.sql("DESCRIBE DATABASE EXTENDED default").show(5, False)

+-------------------------+------------------------------------------+
|database_description_item|database_description_value                |
+-------------------------+------------------------------------------+
|Database Name            |default                                   |
|Comment                  |Default Hive database                     |
|Location                 |gs://dataproc-datalake-demo/hive-warehouse|
|Owner                    |public                                    |
|Properties               |                                          |
+-------------------------+------------------------------------------+



Create a new database called bank_demo_db

In [ ]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS bank_demo_db;
""").show()

++
||
++
++



In [ ]:
spark.sql("DESCRIBE DATABASE EXTENDED bank_demo_db").show(5, False)

+-------------------------+----------------------------------------------------------+
|database_description_item|database_description_value                                |
+-------------------------+----------------------------------------------------------+
|Database Name            |bank_demo_db                                              |
|Comment                  |                                                          |
|Location                 |gs://dataproc-datalake-demo/hive-warehouse/bank_demo_db.db|
|Owner                    |root                                                      |
|Properties               |                                                          |
+-------------------------+----------------------------------------------------------+



Load the CSV file into a Spark Dataframe

In [ ]:
df_bank_marketing = spark \
  .read \
  .option ( "inferSchema" , "true" ) \
  .option ( "header" , "true" ) \
  .csv ( "gs://cloud-ml-tables-data/bank-marketing.csv" )

df_bank_marketing.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Default: string (nullable = true)
 |-- Balance: integer (nullable = true)
 |-- Housing: string (nullable = true)
 |-- Loan: string (nullable = true)
 |-- Contact: string (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Campaign: integer (nullable = true)
 |-- PDays: integer (nullable = true)
 |-- Previous: integer (nullable = true)
 |-- POutcome: string (nullable = true)
 |-- Deposit: integer (nullable = true)



In [ ]:
df_bank_marketing.show(5)

+---+------------+-------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|Age|         Job|MaritalStatus|Education|Default|Balance|Housing|Loan|Contact|Day|Month|Duration|Campaign|PDays|Previous|POutcome|Deposit|
+---+------------+-------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 58|  management|      married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown|      1|
| 44|  technician|       single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown|      1|
| 33|entrepreneur|      married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown|      1|
| 47| blue-collar|      married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown|      1|
| 33|     unknown|  

Run transformations on the data

In [ ]:
## Any transformations on your data can be done at this point

Save the dataframe as a Hive table in Parquet format

In [ ]:
df_bank_marketing.write.mode('overwrite').format("parquet").saveAsTable("bank_demo_db.bank_marketing")

Check that table was created

In [ ]:
spark.sql("SHOW TABLES in bank_demo_db").show()

+------------+--------------+-----------+
|    database|     tableName|isTemporary|
+------------+--------------+-----------+
|bank_demo_db|bank_marketing|      false|
+------------+--------------+-----------+



In [ ]:
spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing").show(100, False)

+----------------------------+-------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                |comment|
+----------------------------+-------------------------------------------------------------------------+-------+
|Age                         |int                                                                      |null   |
|Job                         |string                                                                   |null   |
|MaritalStatus               |string                                                                   |null   |
|Education                   |string                                                                   |null   |
|Default                     |string                                                                   |null   |
|Balance                     |int                                                               

### Compute statistics for columns in table

In [ ]:
spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing Age").show()

+--------------+----------+
|     info_name|info_value|
+--------------+----------+
|      col_name|       Age|
|     data_type|       int|
|       comment|      NULL|
|           min|      NULL|
|           max|      NULL|
|     num_nulls|      NULL|
|distinct_count|      NULL|
|   avg_col_len|      NULL|
|   max_col_len|      NULL|
|     histogram|      NULL|
+--------------+----------+



In [ ]:
cols = "Age, Job, MaritalStatus"
analyzeTableSQL = "ANALYZE TABLE bank_demo_db.bank_marketing COMPUTE STATISTICS FOR COLUMNS Age, Job, MaritalStatus"
spark.sql(analyzeTableSQL).show()

++
||
++
++



In [ ]:
cols = "Age, Job, MaritalStatus"
analyzeTableSQL = "ANALYZE TABLE bank_demo_db.bank_marketing COMPUTE STATISTICS FOR ALL COLUMNS"
spark.sql(analyzeTableSQL).show()

++
||
++
++



In [ ]:
spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing Age").show()

+--------------+----------+
|     info_name|info_value|
+--------------+----------+
|      col_name|       Age|
|     data_type|       int|
|       comment|      NULL|
|           min|        18|
|           max|        95|
|     num_nulls|         0|
|distinct_count|        76|
|   avg_col_len|         4|
|   max_col_len|         4|
|     histogram|      NULL|
+--------------+----------+



In [ ]:
spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing Job").show()

+--------------+----------+
|     info_name|info_value|
+--------------+----------+
|      col_name|       Job|
|     data_type|    string|
|       comment|      NULL|
|           min|      NULL|
|           max|      NULL|
|     num_nulls|         0|
|distinct_count|        11|
|   avg_col_len|        10|
|   max_col_len|        13|
|     histogram|      NULL|
+--------------+----------+



You can now also see how many rows are in the table

In [ ]:
spark.sql("DESCRIBE TABLE EXTENDED bank_demo_db.bank_marketing").show(100, False)

+----------------------------+-------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                |comment|
+----------------------------+-------------------------------------------------------------------------+-------+
|Age                         |int                                                                      |null   |
|Job                         |string                                                                   |null   |
|MaritalStatus               |string                                                                   |null   |
|Education                   |string                                                                   |null   |
|Default                     |string                                                                   |null   |
|Balance                     |int                                                               

Cache the DataFrame in memory 

In [ ]:
data.cache()

DataFrame[Age: int, Job: string, MaritalStatus: string, Education: string, Default: string, Balance: int, Housing: string, Loan: string, Contact: string, Day: int, Month: string, Duration: int, Campaign: int, PDays: int, Previous: int, POutcome: string, Deposit: int]

In [ ]:
data.groupBy("Deposit").count().show()

+-------+-----+
|Deposit|count|
+-------+-----+
|      1|39922|
|      2| 5289|
+-------+-----+

